In [14]:
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from Utilities import *


# Paths
data_path = '/home/rbertille/data/pycharm/ViT_project/pycharm_Geoflow/GeoFlow/Tutorial/Datasets/'
dataset_name = 'Dataset1Dhuge_96tr'
files_path = os.path.join(data_path, dataset_name)

train_folder = glob.glob(f'{files_path}/train/*')
validate_folder = glob.glob(f'{files_path}/validate/*')
test_folder = glob.glob(f'{files_path}/test/*')

class CustomDataset(Dataset):
    def __init__(self, folder, transform=None):
        self.data, self.labels = self.load_data_from_folder(folder)
        self.transform = transform

    def load_data_from_folder(self, folder):
        data = []
        labels = []

        for file_path in folder:
            with h5.File(file_path, 'r') as h5file:
                inputs = h5file['shotgather'][:]
                len_inp = inputs.shape[1]
                half_ind= len_inp//2
                inputs = h5file['shotgather'][:,half_ind:]
                labels_data = h5file['vsdepth'][:]
                
                transform_tensor = transforms.Compose([
                    transforms.ToTensor()
                ])
                
                
                inputs = transform_tensor(inputs)

                data.append(inputs)
                labels.append(labels_data)

        data = np.array(data)


        labels = np.array(labels)
        return data, labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        inputs = self.data[idx]
        labels = self.labels[idx]

        # Convert inputs and labels to Tensors
        inputs = torch.tensor(inputs, dtype=torch.float32)
        labels = torch.tensor(labels, dtype=torch.float32)


        if self.transform:
            inputs = self.transform(inputs)
            
        sample = {'data': inputs, 'label': labels}

        return sample

def create_datasets(data_path, dataset_name):
    train_folder = glob.glob(os.path.join(data_path, dataset_name, 'train', '*'))
    validate_folder = glob.glob(os.path.join(data_path, dataset_name, 'validate', '*'))
    test_folder = glob.glob(os.path.join(data_path, dataset_name, 'test', '*'))

    train_dataset = CustomDataset(train_folder)
    validate_dataset = CustomDataset(validate_folder)
    test_dataset = CustomDataset(test_folder)

    return train_dataset, validate_dataset, test_dataset


train_dataset, validate_dataset, test_dataset= create_datasets(data_path, dataset_name)

train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=True)

In [15]:
#take one sample as example
i=0
sample = train_dataloader.dataset[i]
image = sample['data']
label= sample['label']

In [16]:
#calculate maximum velocity in the whole dataset
max_c = 0
for i in range(len(train_dataloader.dataset)):
    sample = train_dataloader.dataset[i]
    label = sample['label']
    max_label=torch.max(label)
    max_c = np.max([max_c,max_label])
for i in range(len(validate_dataset)):
    sample = validate_dataset[i]
    label = sample['label']
    max_label=torch.max(label)
    max_c = np.max([max_c,max_label])
for i in range(len(test_dataset)):
    sample = test_dataset[i]
    label = sample['label']
    max_label=torch.max(label)
    max_c = np.max([max_c,max_label])
print('max_c:',max_c)


In [17]:
#display shotgather
plt.imshow(image[0].numpy(),aspect='auto',cmap='gray');plt.show()

In [32]:
fmax = 35  # maximum frequency
dt = 0.00002 * 100  # timestep * resampling
src_pos = [3.]  # source position
rec_pos = np.array([4., 5., 6., 7., 8., 9., 10., 11., 12., 13., 14., 15., 16.,
                    17., 18., 19., 20., 21., 22., 23., 24., 25., 26., 27., 28.,
                    29., 30., 31., 32., 33., 34., 35., 36., 37., 38., 39., 40.,
                    41., 42., 43., 44., 45., 46., 47., 48., 49., 50., 51., 52.,
                    53., 54., 55., 56., 57., 58., 59., 60., 61., 62., 63., 64.,
                    65., 66., 67., 68., 69., 70., 71., 72., 73., 74., 75., 76.,
                    77., 78., 79., 80., 81., 82., 83., 84., 85., 86., 87., 88.,
                    89., 90., 91., 92., 93., 94., 95., 96., 97., 98., 99.])  
# receivers positions
dg = np.abs(rec_pos[0] - rec_pos[1])  # receiver spacing
off0 = np.abs(rec_pos[0] - src_pos[0])  # minimum offset
off1 = np.abs(rec_pos[-1] - rec_pos[0])  # maximum offset
ng = rec_pos.shape[-1]  # number of receivers
offmin, offmax = np.min([off0, off1]), np.max([off0, off1])
x = rec_pos - src_pos[0]  # offsets
c = np.logspace(2, np.log10(max_c), num=224)  # phase velocities
print('c:',c)
d= image[0].numpy()

In [33]:
import tensorflow as tf
import math

def phase_shift_sum_base(d, f, x, c, adjoint=False):
    """
    Apply the phase shift summation to a signal in the frequency domain

    :param d: Data with shape [..., nx, nf] if adjoint is False,
              or [..., nc, nt] if adjoint is True.
    :param f: Frequency vector with nf components
    :param x: A 1D array containing the full offset of the nx traces
    :param c: A 1D array containing the nc velocities
    :param adjoint: If True, goes from the velocity-freq to offset-freq domain,
                    if False, goes from offset-freq to velocity-freq domain.

    :return:
        m: Phase-shifted summation of the signal. If adjoint, the dimensions
           is [..., nx, nf], else is [..., nc, nf]
    """

    nd = tf.rank(d)
    inds = tf.concat([tf.ones(nd-1, dtype=tf.int32), [-1]], axis=0)
    f = tf.cast(tf.reshape(f, inds), d.dtype)

    c = tf.cast(c, d.dtype)
    nc = c.shape[-1]

    inds = tf.concat([tf.ones(nd - 2, dtype=tf.int32), [-1, 1]], axis=0)
    x = tf.cast(x, d.dtype)
    nx = x.shape[-1]

    if adjoint:
        c = tf.reshape(c, inds)
        m = tf.TensorArray(d.dtype, size=nx)
        for ix in tf.range(nx):
            i = tf.cast(tf.complex(0.0, -1.0), d.dtype)
            delta = tf.exp(i * 2 * math.pi * f * x[ix] / c)
            m = m.write(ix, tf.reduce_sum(delta * d, axis=-2))
    else:
        x = tf.reshape(x, inds)
        m = tf.TensorArray(d.dtype, size=nc)
        for ic in tf.range(nc):
            i = tf.cast(tf.complex(0.0, 1.0), d.dtype)
            delta = tf.exp(i * 2 * math.pi * f * x / c[ic])
            m = m.write(ic, tf.reduce_sum(delta * d, axis=-2))

    leading = tf.range(1, nd - 1)
    trailing = tf.constant([-1]) + tf.rank(f)
    new_order = tf.concat([leading, [0], trailing], axis=0)

    return tf.transpose(m.stack(), new_order)

def phase_shift_sum(d, f, x, c, adjoint=False):
    """
    Apply the phase shift summation to a signal in the frequency domain,
    and provide a custom gradient.

    :param d: Data with shape [..., nx, nf] if adjoint is False,
              or [..., nc, nt] if adjoint is True.
    :param f: Frequency vector with nf components
    :param x: A 1D array containing the full offset of the nx traces
    :param c: A 1D array containing the nc velocities
    :param adjoint: If True, goes from the velocity-freq to offset-freq domain,
                    if False, goes from offset-freq to velocity-freq domain.

    :return:
        m: Phase-shifted summation of the signal. If adjoint, the dimensions
           is [..., nx, nf], else is [..., nc, nf]
    """
    dout = phase_shift_sum_base(d, f, x, c, adjoint=adjoint)
    return dout

    # Note: The gradient function is omitted here


def linear_radon_freq(d, dt, x, c, fmax=None, norm=False, epsilon=0.001):

    nt = d.shape[-1]
    d_fft = tf.signal.rfft(d)
    if norm: d_fft /= tf.cast(tf.abs(d_fft) + epsilon*tf.math.reduce_max(tf.abs(d_fft)),dtype=tf.complex64)
    fnyq = 1.00 / (nt*dt) * (nt//2+1)
    if fmax is None:
        fmax = fnyq
    if fmax > fnyq:
        raise ValueError("fmax=%f is greater than nyquist=%f"
                         % (fmax, 0.5 / dt))
    f = tf.range(fmax, delta=1.00 / (nt*dt))
    nf = f.shape[-1]

    d_fft = d_fft[..., :nf]

    m = phase_shift_sum(d_fft, f, x, c)

    return m



def dispersion(d, dt, x, c, fmax=None,epsilon=0.001):
    '''
    calculate the dispersion image using phase shift method
    :param d: shotgather
    :param dt: timestep
    :param x: position of the receivers
    :param c: 
    :param fmax: 
    :param epsilon: 
    :return: 
    '''
    return tf.abs(linear_radon_freq(d, dt, x, c, fmax=fmax, norm=True,epsilon=epsilon))



In [34]:
import tensorflow as tf
import math

class PhaseShiftMethod:
    def __init__(self, dt, x, c, fmax=None, epsilon=0.001):
        """
        Initialize the PhaseShiftMethod class with necessary parameters.

        :param dt: Timestep
        :param x: Position of the receivers
        :param c: Array containing the velocities
        :param fmax: Maximum frequency (optional)
        :param epsilon: Small value to avoid division by zero (default=0.001)
        """
        self.dt = dt
        self.x = x
        self.c = c
        self.fmax = fmax
        self.epsilon = epsilon

    def phase_shift_sum_base(self, d, f, adjoint=False):
        """
        Apply the phase shift summation to a signal in the frequency domain.

        :param d: Data with shape [..., nx, nf] if adjoint is False,
                  or [..., nc, nt] if adjoint is True.
        :param f: Frequency vector with nf components
        :param adjoint: If True, goes from the velocity-freq to offset-freq domain,
                        if False, goes from offset-freq to velocity-freq domain.
        :return:
            m: Phase-shifted summation of the signal. If adjoint, the dimensions
               is [..., nx, nf], else is [..., nc, nf]
        """
        nd = tf.rank(d)
        inds = tf.concat([tf.ones(nd-1, dtype=tf.int32), [-1]], axis=0)
        f = tf.cast(tf.reshape(f, inds), d.dtype)

        c = tf.cast(self.c, d.dtype)
        nc = c.shape[-1]

        inds = tf.concat([tf.ones(nd - 2, dtype=tf.int32), [-1, 1]], axis=0)
        x = tf.cast(self.x, d.dtype)
        nx = x.shape[-1]

        if adjoint:
            c = tf.reshape(c, inds)
            m = tf.TensorArray(d.dtype, size=nx)
            for ix in tf.range(nx):
                i = tf.cast(tf.complex(0.0, -1.0), d.dtype)
                delta = tf.exp(i * 2 * math.pi * f * x[ix] / c)
                m = m.write(ix, tf.reduce_sum(delta * d, axis=-2))
        else:
            x = tf.reshape(x, inds)
            m = tf.TensorArray(d.dtype, size=nc)
            for ic in tf.range(nc):
                i = tf.cast(tf.complex(0.0, 1.0), d.dtype)
                delta = tf.exp(i * 2 * math.pi * f * x / c[ic])
                m = m.write(ic, tf.reduce_sum(delta * d, axis=-2))

        leading = tf.range(1, nd - 1)
        trailing = tf.constant([-1]) + tf.rank(f)
        new_order = tf.concat([leading, [0], trailing], axis=0)

        return tf.transpose(m.stack(), new_order)

    def phase_shift_sum(self, d, f, adjoint=False):
        """
        Apply the phase shift summation to a signal in the frequency domain,
        and provide a custom gradient.

        :param d: Data with shape [..., nx, nf] if adjoint is False,
                  or [..., nc, nt] if adjoint is True.
        :param f: Frequency vector with nf components
        :param adjoint: If True, goes from the velocity-freq to offset-freq domain,
                        if False, goes from offset-freq to velocity-freq domain.
        :return:
            m: Phase-shifted summation of the signal. If adjoint, the dimensions
               is [..., nx, nf], else is [..., nc, nf]
        """
        return self.phase_shift_sum_base(d, f, adjoint=adjoint)

    def linear_radon_freq(self, d, norm=False):
        """
        Compute the linear Radon transform in the frequency domain.

        :param d: Input data array
        :param norm: Whether to normalize the Fourier transform (default=False)
        :return:
            m: Transformed data in the frequency domain.
        """
        nt = d.shape[-1]
        d_fft = tf.signal.rfft(d)
        if norm:
            d_fft /= tf.cast(tf.abs(d_fft) + self.epsilon * tf.math.reduce_max(tf.abs(d_fft)), dtype=tf.complex64)

        fnyq = 1.00 / (nt * self.dt) * (nt // 2 + 1)
        if self.fmax is None:
            self.fmax = fnyq
        if self.fmax > fnyq:
            raise ValueError(f"fmax={self.fmax} is greater than nyquist={0.5 / self.dt}")

        f = tf.range(self.fmax, delta=1.00 / (nt * self.dt))
        nf = f.shape[-1]

        d_fft = d_fft[..., :nf]

        return self.phase_shift_sum(d_fft, f)

    def dispersion(self, d):
        """
        Calculate the dispersion image using phase shift method.

        :param d: Shot gather data
        :return:
            Dispersion image in the frequency domain.
        """
        return tf.abs(self.linear_radon_freq(d, norm=True))

In [35]:
print('shape of the shotgather:',d.shape)
disp = dispersion(d.T, dt, x, c, epsilon=1e-6,fmax=fmax).numpy().T
#normalize the dispersion image
disp = (disp - np.min(disp)) / (np.max(disp) - np.min(disp))

In [36]:
#display the dispersion image
plt.imshow(disp,aspect='auto',cmap='jet')
#labels
plt.xlabel('phase velocity')
# put the right grafduations on x axe, that correspond to linspace(min_c,max_c) with 5 ticks ONLY, with vertical ticks rounded to near integer
plt.xticks(np.linspace(0,disp.shape[1]-1,5).astype(int),np.round(np.linspace(np.min(c),np.max(c),5)).astype(int))
plt.ylabel('frequence')
plt.title('Dispersion image')
#add colorbar
plt.colorbar()
plt.show()

In [37]:
#display in first column the shotgather, in the second column the dispersion image and in the third column the Vsdepth
n=5
for i in range(n):
    #take a random sample as example
    j=np.random.randint(0,len(train_dataloader.dataset))
    sample = train_dataloader.dataset[j]
    image = sample['data']
    label= sample['label']
    #display the shotgather
    plt.figure(figsize=(15,5))
    plt.subplot(1,3,1)
    plt.imshow(image[0].numpy(),aspect='auto',cmap='gray')
    plt.title('Shotgather')
    #calculate the dispersion
    disp = dispersion(image[0].numpy().T, dt, x, c, epsilon=1e-6,fmax=35).numpy().T
    disp = (disp - np.min(disp)) / (np.max(disp) - np.min(disp))
    #display the dispersion image
    plt.subplot(1,3,2)
    plt.imshow(disp,aspect='auto',cmap='jet')
    plt.xticks(np.linspace(0,disp.shape[1]-1,5).astype(int),np.round(np.linspace(np.min(c),np.max(c),5)).astype(int))
    plt.colorbar()
    plt.title('Dispersion image')
    #display the Vsdepth
    plt.subplot(1,3,3)
    plt.plot(label, range(len(label)))
    plt.gca().invert_yaxis()

    plt.title('Vsdepth')
    plt.show()

In [ ]:
plt.figure(figsize=(15,7))
sample = train_dataloader.dataset[16]
image = sample['data']
label= sample['label']

# convert the image from grid points into real units
dt = 0.00002 * 100  # dt * resampling
time_vector = np.arange(image.shape[1]) * dt
print('time vector len:',len(time_vector))
nb_traces = image.shape[2]
print('nb traces:',nb_traces)
dz = 0.25
# print('label shape:',label.shape)
depth_vector = np.arange(label.shape[0]) * dz

plt.subplot(1,2,1)
plt.imshow(image[0].numpy(),aspect='auto',cmap='gray',extent=[0, nb_traces, time_vector[-1], time_vector[0]])
plt.title('données sismiques')
plt.xlabel('traces')
plt.ylabel('temps (s)')

plt.subplot(1,2,2)
disp = dispersion(image[0].numpy().T, dt, x, c, epsilon=1e-6,fmax=35).numpy().T
#normalize the dispersion image
disp = (disp - np.min(disp)) / (np.max(disp) - np.min(disp))
plt.imshow(disp,aspect='auto',cmap='jet')
plt.xticks(np.linspace(0,disp.shape[1]-1,5).astype(int),np.round(np.linspace(np.min(c),np.max(c),5)).astype(int))
plt.colorbar(label='amplitude normalisée')
plt.title('Image de dispersion')
plt.xlabel('vitesse de phase (m/s)')
plt.ylabel('fréquence (Hz)')

plt.savefig('figures/example_dispersion.png',format='png')
plt.show()

#plot only the dispersion image
plt.figure(figsize=(6,6))
plt.imshow(disp,aspect='auto',cmap='jet')
plt.xticks(np.linspace(0,disp.shape[1]-1,5).astype(int),np.round(np.linspace(np.min(c),np.max(c),5)).astype(int))
plt.colorbar(label='amplitude normalisée')
plt.title('Image de dispersion')
plt.xlabel('vitesse de phase (m/s)')
plt.ylabel('fréquence (Hz)')
plt.savefig('figures/example_dispersion_only.png',format='png')
plt.show()